# Produccion electrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a que aumenten la produccion electrica de energias renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto es prácticamente inmediato vamos a centrarnos en la produccion de energia hidroelectrica.

En base a esto tenemos varias cuestiones que queremos despejar:
- Cuanto tarda en notarse en la generación eléctrica de fuentes de energía renovables los fenomenos metereológicos.
- Que cantidad de precipitaciones son necesarias para aumentar la generación de energia eléctica.
- Existe realmente una relación entre la metereologia y la producción electrica.


Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Se va a dividir el estudio por provincias, comunidades autonomas y a nivel estatal.

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request



In [8]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import requests
import time

# Leemos la clave de la API de AEMET desde un fichero ubicado en la misma ruta que este notebook
with open('API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()

    

In [9]:
# Obtenemos la lista de estaciones de medicion de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE)


In [10]:
# Lectura de datos de AEMET


# Definimos funciones que vamos a utilizar para leer los datos de aement
def estaciones_prov (prov,lista_estaciones):
    '''Dada una provincia y una lista de diccionarios de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    lista_id=[]
    for estacion in lista_estaciones:
        if estacion['provincia']==prov.upper():
            lista_id.append(estacion['indicativo'])
    return lista_id

help(estaciones_prov)


def lectura_dias(date_ini,date_end,estaciones):
    '''Dado un json del objeto Estacion, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in estaciones:
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicativo'])
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(0.1) # para evitar errores por nº de lecturas.
    return valores_diarios;

help(lectura_dias)

# Pasamos los datos ha dataframes para su procesado
estaciones=pd.DataFrame(info_estaciones)

date_ini="2020-12-31T00:00:00UTC"
date_end="2020-12-31T00:00:00UTC"

wheather=pd.DataFrame(lectura_dias(date_ini,date_end,info_estaciones))

print(max(wheather['fecha']),min(wheather['fecha']))


#Rellenamos los NaN por 0 para precipitaciones. Y para temperatura los NaN por la termperatura del dia anterior


Help on function estaciones_prov in module __main__:

estaciones_prov(prov, lista_estaciones)
    Dada una provincia y una lista de diccionarios de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.

Help on function lectura_dias in module __main__:

lectura_dias(date_ini, date_end, estaciones)
    Dado un json del objeto Estacion, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.

2020-12-22 2020-12-01


In [11]:
# Obtenemos los datos de REE a traves de su API.

http=urllib3.PoolManager()

URL='https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2019-01-01T00:00&end_date=2019-01-01T01:00&time_trunc=day'
ree=requests.get(URL)

type(ree)


#ree=pd.DataFrame(request)

# Paso el resultado a str
#type(request.data.decode())

     
# Paso el str a json para poder pasarlo a Dataframe
#prueba=json.loads(request.data)


requests.models.Response